In [40]:
%pip install geobr

Note: you may need to restart the kernel to use updated packages.


In [41]:
import requests
from pathlib import Path
import geopandas as gpd
from typing import List, Dict, Optional
from geobr import read_municipality

In [42]:
class BrazilianMunicipalityShapefiles:
    
    def __init__(self, download_dir: str = "files/municipal_shapefiles"):
        self.download_dir = Path(download_dir)
        self.download_dir.mkdir(parents=True, exist_ok=True)
        self.api_base = "https://servicodados.ibge.gov.br/api/v1"
        self._states_cache = None
        self._municipalities_cache = {}
    
    def get_all_states(self) -> List[Dict]:
        if self._states_cache is not None:
            return self._states_cache
        
        url = f"{self.api_base}/localidades/estados"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        states = sorted(response.json(), key=lambda x: x['id'])
        self._states_cache = states
        return states
    
    def get_municipalities_by_state(self, state_code: Optional[str] = None, 
                                     state_sigla: Optional[str] = None) -> List[Dict]:
        if not state_code and not state_sigla:
            raise ValueError("Either state_code or state_sigla must be provided")
        
        if state_sigla and not state_code:
            states = self.get_all_states()
            state = next((s for s in states if s['sigla'].upper() == state_sigla.upper()), None)
            if not state:
                raise ValueError(f"State '{state_sigla}' not found")
            state_code = state['id']
        
        if state_code in self._municipalities_cache:
            return self._municipalities_cache[state_code]
        
        url = f"{self.api_base}/localidades/estados/{state_code}/municipios"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        municipalities = sorted(response.json(), key=lambda x: x['nome'])
        self._municipalities_cache[state_code] = municipalities
        return municipalities
    
    def download_municipality_shapefile(self, municipality_code, 
                                        municipality_name: Optional[str] = None,
                                        state_code: Optional[str] = None,
                                        year: int = 2020) -> Optional[Path]:
        municipality_code = str(municipality_code)
        
        if municipality_name:
            safe_name = "".join(c for c in municipality_name if c.isalnum() or c in (' ', '-', '_')).strip()
            mun_dir = self.download_dir / f"{municipality_code}_{safe_name}"
        else:
            mun_dir = self.download_dir / municipality_code
        
        mun_dir.mkdir(parents=True, exist_ok=True)
        
        existing_shp = list(mun_dir.glob("*.shp"))
        if existing_shp:
            return mun_dir
        
        try:
            mun_code_int = int(municipality_code)
            gdf = read_municipality(code_muni=mun_code_int, year=year)
            
            if len(gdf) > 0:
                output_shp = mun_dir / f"{municipality_code}.shp"
                gdf.to_file(output_shp)
                return mun_dir
        except Exception:
            pass
        
        return None
    
    def download_municipality_by_name(self, municipality_name: str, 
                                      state_sigla: str) -> Optional[Path]:
        municipalities = self.get_municipalities_by_state(state_sigla=state_sigla)
        municipality = next(
            (mun for mun in municipalities if municipality_name.upper() in mun['nome'].upper()),
            None
        )
        
        if not municipality:
            return None
        
        return self.download_municipality_shapefile(
            municipality_code=str(municipality['id']),
            municipality_name=municipality['nome']
        )
    
    def print_states(self):
        states = self.get_all_states()
        print("\n" + "="*60)
        print("Brazilian States:")
        print("="*60)
        for state in states:
            print(f"  {state['sigla']:2s} - {state['nome']:30s} (Code: {state['id']})")
        print("="*60)
    
    def print_municipalities(self, state_sigla: str):
        municipalities = self.get_municipalities_by_state(state_sigla=state_sigla)
        print(f"\n{'='*60}")
        print(f"Municipalities in {state_sigla}:")
        print("="*60)
        for i, mun in enumerate(municipalities, 1):
            print(f"  {i:3d}. {mun['nome']:40s} (Code: {mun['id']})")
        print("="*60)


# Brazilian Municipal Shapefiles Downloader

This notebook provides a class to download shapefiles for Brazilian municipalities from IBGE.

## Features:
- Get all Brazilian states
- Get all municipalities in a given state
- Download shapefiles for specific municipalities


In [43]:
downloader = BrazilianMunicipalityShapefiles()


## Example 1: Get all Brazilian states


In [44]:
states = downloader.get_all_states()
downloader.print_states()
print(f"\nTotal states: {len(states)}")
print(f"First state: {states[0]}")



Brazilian States:
  RO - Rondônia                       (Code: 11)
  AC - Acre                           (Code: 12)
  AM - Amazonas                       (Code: 13)
  RR - Roraima                        (Code: 14)
  PA - Pará                           (Code: 15)
  AP - Amapá                          (Code: 16)
  TO - Tocantins                      (Code: 17)
  MA - Maranhão                       (Code: 21)
  PI - Piauí                          (Code: 22)
  CE - Ceará                          (Code: 23)
  RN - Rio Grande do Norte            (Code: 24)
  PB - Paraíba                        (Code: 25)
  PE - Pernambuco                     (Code: 26)
  AL - Alagoas                        (Code: 27)
  SE - Sergipe                        (Code: 28)
  BA - Bahia                          (Code: 29)
  MG - Minas Gerais                   (Code: 31)
  ES - Espírito Santo                 (Code: 32)
  RJ - Rio de Janeiro                 (Code: 33)
  SP - São Paulo                      (Code: 35)
 

## Example 2: Get all municipalities in a state


In [45]:
municipalities_pr = downloader.get_municipalities_by_state(state_sigla="PR")
downloader.print_municipalities("SP")
print(f"\nTotal municipalities in SP: {len(municipalities_pr)}")
print(f"First municipality: {municipalities_pr[0]}")



Municipalities in SP:
    1. Adamantina                               (Code: 3500105)
    2. Adolfo                                   (Code: 3500204)
    3. Aguaí                                    (Code: 3500303)
    4. Agudos                                   (Code: 3500709)
    5. Alambari                                 (Code: 3500758)
    6. Alfredo Marcondes                        (Code: 3500808)
    7. Altair                                   (Code: 3500907)
    8. Altinópolis                              (Code: 3501004)
    9. Alto Alegre                              (Code: 3501103)
   10. Alumínio                                 (Code: 3501152)
   11. Alvinlândia                              (Code: 3501509)
   12. Americana                                (Code: 3501608)
   13. Amparo                                   (Code: 3501905)
   14. Américo Brasiliense                      (Code: 3501707)
   15. Américo de Campos                        (Code: 3501806)
   16. Analândia 

## Example 3: Download shapefile by municipality code


In [46]:
abatia_path = downloader.download_municipality_shapefile(
    municipality_code="4100103",
    municipality_name="Abatiá"
)

print('path: ', abatia_path)

if abatia_path:
    gdf = gpd.read_file(list(abatia_path.glob("*.shp"))[0])
    print(f"Shapefile: {abatia_path}")
    print(f"Geometry: {gdf.geometry.type[0]}")
    print(f"CRS: {gdf.crs}")
    print(f"Bounds: {gdf.total_bounds}")


path:  files\municipal_shapefiles\4100103_Abatiá
Shapefile: files\municipal_shapefiles\4100103_Abatiá
Geometry: Polygon
CRS: EPSG:4674
Bounds: [-50.48103856 -23.38284845 -50.19146793 -23.22442889]


C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(


In [47]:
campinas_path = downloader.download_municipality_by_name(
    municipality_name="Campinas",
    state_sigla="SP"
)

if campinas_path:
    gdf = gpd.read_file(list(campinas_path.glob("*.shp"))[0])
    print(f"Shapefile: {campinas_path}")
    print(f"Municipality: {gdf.iloc[0] if len(gdf) > 0 else 'N/A'}")


Shapefile: files\municipal_shapefiles\3509502_Campinas
Municipality: code_muni                                             3509502.0
name_muni                                              Campinas
code_state                                                 35.0
abbrev_sta                                                   SP
name_state                                            São Paulo
code_regio                                                  3.0
name_regio                                              Sudeste
geometry      POLYGON ((-46.945609909999995 -22.738938733000...
Name: 0, dtype: object


C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(


## Example 5: Batch download multiple municipalities


In [48]:
municipalities = downloader.get_municipalities_by_state(state_sigla="PR")

for i, mun in enumerate(municipalities, 1):
    print(f"[{i}/{len(municipalities)}] {mun['nome']}...")
    path = downloader.download_municipality_shapefile(
        municipality_code=mun['id'],
        municipality_name=mun['nome']
    )
    print(f"  {'Success' if path else 'Failed'}\n")


[1/399] Abatiá...
  Success

[2/399] Adrianópolis...
  Success

[3/399] Agudos do Sul...
  Success

[4/399] Almirante Tamandaré...
  Success

[5/399] Altamira do Paraná...
  Success

[6/399] Alto Paraná...
  Success

[7/399] Alto Paraíso...
  Success

[8/399] Alto Piquiri...
  Success

[9/399] Altônia...
  Success

[10/399] Alvorada do Sul...
  Success

[11/399] Amaporã...
  Success

[12/399] Ampére...
  Success

[13/399] Anahy...
  Success

[14/399] Andirá...
  Success

[15/399] Antonina...
  Success

[16/399] Antônio Olinto...
  Success

[17/399] Apucarana...
  Success

[18/399] Arapongas...
  Success

[19/399] Arapoti...
  Success

[20/399] Arapuã...


C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(
C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\L

  Success

[21/399] Araruna...
  Success

[22/399] Araucária...
  Success

[23/399] Ariranha do Ivaí...
  Success

[24/399] Assaí...
  Success

[25/399] Assis Chateaubriand...
  Success

[26/399] Astorga...
  Success

[27/399] Atalaia...
  Success

[28/399] Balsa Nova...


c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(
C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-

  Success

[29/399] Bandeirantes...


...

C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(
C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\L

  Success

[324/399] Santa Helena...
  Success

[325/399] Santa Inês...
  Success

[326/399] Santa Isabel do Ivaí...
  Success

[327/399] Santa Izabel do Oeste...
  Success

[328/399] Santa Lúcia...
  Success

[329/399] Santa Maria do Oeste...
  Success

[330/399] Santa Mariana...
  Success

[331/399] Santa Mônica...
  Success

[332/399] Santa Tereza do Oeste...
  Success

[333/399] Santa Terezinha de Itaipu...
  Success

[334/399] Santana do Itararé...
  Success

[335/399] Santo Antônio da Platina...
  Success

[336/399] Santo Antônio do Caiuá...
  Success

[337/399] Santo Antônio do Paraíso...
  Success

[338/399] Santo Antônio do Sudoeste...
  Success

[339/399] Santo Inácio...
  Success

[340/399] Sapopema...
  Success

[341/399] Sarandi...
  Success

[342/399] Saudade do Iguaçu...
  Success

[343/399] Sengés...
  Success

[344/399] Serranópolis do Iguaçu...
  Success

[345/399] Sertaneja...
  Success

[346/399] Sertanópolis...
  Success

[347/399] Siqueira Campos...
  Success

[34

c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(
C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-

  Success

[352/399] São Jorge do Ivaí...
  Success

[353/399] São Jorge do Patrocínio...
  Success

[354/399] São José da Boa Vista...
  Success

[355/399] São José das Palmeiras...
  Success

[356/399] São José dos Pinhais...
  Success

[357/399] São João...
  Success

[358/399] São João do Caiuá...
  Success

[359/399] São João do Ivaí...
  Success

[360/399] São João do Triunfo...
  Success

[361/399] São Manoel do Paraná...
  Success

[362/399] São Mateus do Sul...
  Success

[363/399] São Miguel do Iguaçu...
  Success

[364/399] São Pedro do Iguaçu...
  Success

[365/399] São Pedro do Ivaí...
  Success

[366/399] São Pedro do Paraná...
  Success

[367/399] São Sebastião da Amoreira...
  Success

[368/399] São Tomé...
  Success

[369/399] Tamarana...
  Success

[370/399] Tamboara...
  Success

[371/399] Tapejara...
  Success

[372/399] Tapira...
  Success

[373/399] Teixeira Soares...
  Success

[374/399] Telêmaco Borba...
  Success

[375/399] Terra Boa...
  Success

[376/399] Ter

C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'abbrev_state' to 'abbrev_sta'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'code_region' to 'code_regio'
  ogr_write(
c:\Users\heigt\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'name_region' to 'name_regio'
  ogr_write(
C:\Users\heigt\AppData\Local\Temp\ipykernel_10100\2736966807.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)
c:\Users\heigt\AppData\Local\Programs\Python\Python312\L